# Data Processing

In [1]:
import os
import pandas as pd
import numpy as np

pd.set_option('display.max_columns', None)

In [2]:
# Define la ruta a la carpeta donde se guardan los archivos
output_dir = "../data/scrapped"

try:
    # 1. Lista todos los archivos en la carpeta
    all_files = os.listdir(output_dir)

    # 2. Filtra solo los archivos que terminan en '.parquet'
    parquet_files = [os.path.join(output_dir, f) for f in all_files if f.endswith('.parquet')]

    # 3. Verifica si hay archivos para procesar
    if not parquet_files:
        print("No se encontraron archivos .parquet en la carpeta especificada.")
    else:
        print(f"Se encontraron {len(parquet_files)} archivos .parquet. Concatenando...")

        # 4. Lee cada archivo Parquet en un DataFrame y los guarda en una lista
        list_of_dfs = [pd.read_parquet(f) for f in parquet_files]

        # 5. Concatena todos los DataFrames de la lista
        combined_df = pd.concat(list_of_dfs, ignore_index=True)

        print("¡Concatenación exitosa! Los archivos se han unido en un solo DataFrame.")
        print("Se ha creado un DataFrame con las siguientes dimensiones:")
        print(f"Filas: {combined_df.shape[0]}, Columnas: {combined_df.shape[1]}")

except FileNotFoundError:
    print("Error: No se encontró la carpeta especificada.")
    print(f"Por favor, revisa que la carpeta '{output_dir}' exista.")
except ImportError:
    print("Error: La librería 'pyarrow' o 'fastparquet' no está instalada.")
    print("Para leer archivos .parquet, necesitas instalar una de estas librerías. Puedes usar el siguiente comando: pip install pyarrow")
except Exception as e:
    print(f"Ocurrió un error inesperado: {e}")

Se encontraron 378 archivos .parquet. Concatenando...
¡Concatenación exitosa! Los archivos se han unido en un solo DataFrame.
Se ha creado un DataFrame con las siguientes dimensiones:
Filas: 9033, Columnas: 35


## Tipos de datos

In [3]:
combined_df.head(3)

,IdJugador,IdClub,IdEquipo,Nombre,NombreCompleto,Puntos,ReboteDefensivo,ReboteOfensivo,RebotesTotales,Asistencias,Recuperaciones,Perdidas,TaponCometido,TaponRecibido,FaltaCometida,FaltaRecibida,Valoracion,TiempoJuego,CincoInicial,equipo,TirosDosAciertos,TirosDosFallos,TirosTresAciertos,TirosTresFallos,TirosLibresAciertos,TirosLibresFallos,plus_minus,posesiones_consumidas,posesiones_jugadas,rebote_of_disp,rebote_def_disp,puntos_q4_y_prorroga,puntos_clutch,posesiones_estimadas,partido_key
0,96264,1932,0,"ARIAS, J.","ARIAS, JUAN PABLO",0,0,0,0,0,0,0,0,0,0,0,0,00:00,False,ARGENTINO (J),0,0,0,0,0,0,0,0,0,0,0,0,0,0.00,ARGENTINO (J) vs ATENAS (C) (002/04/2025 21:00)
1,96269,1932,0,"CAPELLI, S.","CAPELLI, SANTIAGO",0,1,0,1,1,0,0,0,0,1,0,1,06:05,False,ARGENTINO (J),0,0,0,0,0,0,0,0,0,0,0,0,0,0.00,ARGENTINO (J) vs ATENAS (C) (002/04/2025 21:00)
2,323790,1932,0,"FRONTERA, R.","FRONTERA, RAMIRO JEREMIAS",5,6,0,6,2,1,1,1,2,3,2,3,24:02,False,ARGENTINO (J),2,0,0,0,1,0,-21,11,48,30,15,0,0,3.44,ARGENTINO (J) vs ATENAS (C) (002/04/2025 21:00)


In [4]:
# Convertir la columna 'TiempoJuego' de string "MM:SS" a segundos como entero
def tiempo_a_segundos(tiempo_str):
    try:
        minutos, segundos = map(int, tiempo_str.split(":"))
        return minutos * 60 + segundos
    except Exception:
        return 0

combined_df["TiempoJuego_seg"] = combined_df["TiempoJuego"].apply(tiempo_a_segundos)
combined_df["TiempoJuego_min"] = combined_df["TiempoJuego_seg"] / 60

In [14]:
combined_df.shape

(9033, 40)

In [6]:
categorical_cols=['IdJugador', 'IdClub', 'IdEquipo','Nombre', 'NombreCompleto', 'equipo', 'partido_key']
numerical_cols=['Puntos', 'ReboteDefensivo','ReboteOfensivo', 'RebotesTotales', 'Asistencias', 'Recuperaciones',
                'Perdidas', 'TaponCometido', 'TaponRecibido', 'FaltaCometida','FaltaRecibida', 'Valoracion', 
                'TirosDosAciertos', 'TirosDosFallos','TirosTresAciertos', 'TirosTresFallos', 'TirosLibresAciertos',
                'TirosLibresFallos', 'plus_minus', 'posesiones_consumidas','posesiones_jugadas', 'rebote_of_disp', 
                'rebote_def_disp','puntos_q4_y_prorroga', 'puntos_clutch', 'posesiones_estimadas','TiempoJuego_seg','TiempoJuego_min']

In [7]:
# Convertir columnas categóricas a 'category' y numéricas a 'float' (o 'int' si no hay NaN)
for col in categorical_cols:
    if col in combined_df.columns:
        combined_df[col] = combined_df[col].astype('category')

for col in numerical_cols:
    if col in combined_df.columns:
        combined_df[col] = combined_df[col].astype(float)


## Nacionalidad de Jugadores

In [8]:
extranjeros_data = [
    {"NombreCompleto": "CHACON TIRADO, MARCOS", "nacionalidad": "Cubano"},
    {"NombreCompleto": "MILLER, TAVARIO EARNEST PTRISTIAN", "nacionalidad": "Bahamense"},
    {"NombreCompleto": "RAMIREZ ALCANTARA, KELVIN LEANDRO", "nacionalidad": "Dominicano"},
    {"NombreCompleto": "OWENS, DEMARCO RASHAD", "nacionalidad": "Estadounidense"},
    {"NombreCompleto": "THOMAS JR, MARCUS WILEY", "nacionalidad": "Estadounidense"},
    {"NombreCompleto": "VORHEES, WILLIAM LEONARD-DEUBLER", "nacionalidad": "Estadounidense"},
    {"NombreCompleto": "FERGUSON, ROMEAO VENILL", "nacionalidad": "Estadounidense"}, 
    {"NombreCompleto": "LOCKETT, PHILLIP DOMINIQUE DANIEL", "nacionalidad": "Estadounidense"},
    {"NombreCompleto": "CLARKE, CHRISTOPHER ASHTON", "nacionalidad": "Estadounidense"},
    {"NombreCompleto": "THORNTON, WILLIE ALFORD", "nacionalidad": "Estadounidense"},
    {"NombreCompleto": "DANIELS, TRAVIS DORREL", "nacionalidad": "Estadounidense"},
    {"NombreCompleto": "DIGGS, AVERY GERELL", "nacionalidad": "Estadounidense"},
    {"NombreCompleto": "BECTON, REGINALD GEQUAN", "nacionalidad": "Estadounidense"},
    {"NombreCompleto": "CARRERAS PEGUERO, XAVIER MANUEL", "nacionalidad": "Dominicano"},
    {"NombreCompleto": "THOMAS III, CHARLES PRICE", "nacionalidad": "Estadounidense"},
    {"NombreCompleto": "KRAMER, KELBY JOHN", "nacionalidad": "Estadounidense"},
    {"NombreCompleto": "GIVENS, SAMUEL JAMAL", "nacionalidad": "Estadounidense"},
    {"NombreCompleto": "BOWIE JR, JULIUS R", "nacionalidad": "Estadounidense"},
    {"NombreCompleto": "CALFANI PERSINCULA, MATHIAS KENY", "nacionalidad": "Uruguayo"},
    {"NombreCompleto": "MORRISON, DOMINIQUE MONTEL", "nacionalidad": "Estadounidense"},
    {"NombreCompleto": "ALEXANDER, QUINTIN IMMANUEL", "nacionalidad": "Estadounidense"},
    {"NombreCompleto": "CARDENAS ZAMORA, JORDAN ISRAEL", "nacionalidad": "Ecuatoriano"},
    {"NombreCompleto": "ASCANIO SOLORZANO, JOSE GREGORIO", "nacionalidad": "Venezolano"},
    {"NombreCompleto": "WALTON, ZACHERY CHRISTOPHER", "nacionalidad": "Estadounidense"},
    {"NombreCompleto": "THOMAS, DISCHON KYIR", "nacionalidad": "Estadounidense"},
    {"NombreCompleto": "PRIDDY, NATHAN WAYNE", "nacionalidad": "Estadounidense"},
    {"NombreCompleto": "WHITFIELD III, ROBERT JAMARCUS", "nacionalidad": "Estadounidense"},
    {"NombreCompleto": "HORTON, KENNETH WILCHER", "nacionalidad": "Estadounidense"},
    {"NombreCompleto": "HOLT, EMMITT DWIGHT", "nacionalidad": "Estadounidense"},
    {"NombreCompleto": "JENKINS, JALEN KEMAL", "nacionalidad": "Estadounidense"},
    {"NombreCompleto": "SANDERS, NAKIE GERALD", "nacionalidad": "Estadounidense"},
    {"NombreCompleto": "BOND JR, TIMOTHY LAMONT", "nacionalidad": "Estadounidense"},
    {"NombreCompleto": "ROBINSON, BRANDON LAMAR", "nacionalidad": "Estadounidense"},
    {"NombreCompleto": "OBENG MENSAH, YAW", "nacionalidad": "Canadiense"},
    {"NombreCompleto": "TROCHA MORELOS, TONNY JOSE", "nacionalidad": "Colombiano"},
    {"NombreCompleto": "CRAION JR, MICHAEL JOE", "nacionalidad": "Estadounidense"},
    {"NombreCompleto": "BELL, RANDY TYREE", "nacionalidad": "Estadounidense"},
    {"NombreCompleto": "DANIELS, DEANDRE MARTISE", "nacionalidad": "Estadounidense"},
    {"NombreCompleto": "REESE V, JAMES LANARD", "nacionalidad": "Estadounidense"},
    {"NombreCompleto": "WATSON, EARL OVREL", "nacionalidad": "Estadounidense"},
    {"NombreCompleto": "ROQUE MARTINEZ, ROMARIO JOSE", "nacionalidad": "Colombiano"},
    {"NombreCompleto": "PAYTON CLOTTEY, EMMANUEL TRAVON", "nacionalidad": "Estadounidense"},
    {"NombreCompleto": "FIELDS, CALEB JOSEPH", "nacionalidad": "Estadounidense"},
    {"NombreCompleto": "ANDERSON, ALPHONSO JORDAN", "nacionalidad": "Estadounidense"},
    {"NombreCompleto": "BASTARDO JOSE, RAYMON SCARLIN", "nacionalidad": "Dominicano"},
    {"NombreCompleto": "BANYARD, NICHOLAS RYAN", "nacionalidad": "Estadounidense"},
    {"NombreCompleto": "MAXWELL, DU'VAUGHN ELISHA", "nacionalidad": "Estadounidense"},
    {"NombreCompleto": "RUIZ RUIZ, JOSE DANIEL", "nacionalidad": "Colombiano"},
    {"NombreCompleto": "FUNDORA ARRECHAVALETA, YASMANY	", "nacionalidad": "Cubano"}, 
    {"NombreCompleto": "KRAYEM, OMAR NABIL", "nacionalidad": "Palestino"}, # Estadounidense
    {"NombreCompleto": "WALLACE, DEVANTE RASHAD-KEITH", "nacionalidad": "Estadounidense"},
    {"NombreCompleto": "STOKES, KAMAU THUTMOSES", "nacionalidad": "Estadounidense"},
    {"NombreCompleto": "LOWERY, DISHON LURELL", "nacionalidad": "Estadounidense"},
    {"NombreCompleto": "ALI, PRINCE ADAMS", "nacionalidad": "Estadounidense"},
    {"NombreCompleto": "BONIZIOLI HONORATO, ITALO	", "nacionalidad": "Brasileño"}, 
    {"NombreCompleto": "CARTER, MYLES JUSTIN", "nacionalidad": "Estadounidense"},
    {"NombreCompleto": "HOOPER, CHRISTOPHER JALEEL", "nacionalidad": "Estadounidense"},
    {"NombreCompleto": "OPOKU, NANA KWASI HYEAKURO", "nacionalidad": "Estadounidense"},
    {"NombreCompleto": "HAMILTON, ISAAC BRANDON", "nacionalidad": "Estadounidense"},
    {"NombreCompleto": "NAYLOR, CAMERON LEVELE", "nacionalidad": "Estadounidense"},
    {"NombreCompleto": "MADRIGAL RENTERIA, JUAN ESTEBAN", "nacionalidad": "Colombiano"},
    {"NombreCompleto": "NIEBLES HERRERA, EDWIN JOSE", "nacionalidad": "Colombiano"},
    {"NombreCompleto": "PETRI, NICHOLAS", "nacionalidad": "Estadounidense"},
    {"NombreCompleto": "GUERRA CAÑATE, YEFERSON ANTONIO", "nacionalidad": "Venezolano"},
    {"NombreCompleto": "GARCIA GUERRERO, JORGE LUIS", "nacionalidad": "Venezolano"}, 
    {"NombreCompleto": "HERNANDEZ, MANUEL ALONSO", "nacionalidad": "Estadounidense"}, # Mexicano
]

df_extranjeros = pd.DataFrame(extranjeros_data)
df_extranjeros['nacionalidad'].value_counts()

nacionalidad
Estadounidense    47
Colombiano         5
Dominicano         3
Venezolano         3
Cubano             2
Bahamense          1
Uruguayo           1
Ecuatoriano        1
Canadiense         1
Palestino          1
Brasileño          1
Name: count, dtype: int64

In [10]:
# Eliminar espacios en blanco en 'NombreCompleto' para ambos DataFrames
df_extranjeros['NombreCompleto'] = df_extranjeros['NombreCompleto'].str.strip()
combined_df['NombreCompleto'] = combined_df['NombreCompleto'].str.strip()

# Realizar el left join para agregar la nacionalidad
combined_df = combined_df.merge(df_extranjeros, on='NombreCompleto', how='left')

# Rellenar los valores nulos con 'Argentino'
combined_df['nacionalidad'] = combined_df['nacionalidad'].fillna('Argentino')

In [11]:
# Extraer el rival
combined_df['rival'] = combined_df['partido_key'].str.extract(r'vs\s(.+?)\s\(')
combined_df['fecha_hora'] = combined_df['partido_key'].str[-20:]
combined_df['fecha_hora'] = combined_df['fecha_hora'].str[3:13]

In [13]:
combined_df.head(3)

,IdJugador,IdClub,IdEquipo,Nombre,NombreCompleto,Puntos,ReboteDefensivo,ReboteOfensivo,RebotesTotales,Asistencias,Recuperaciones,Perdidas,TaponCometido,TaponRecibido,FaltaCometida,FaltaRecibida,Valoracion,TiempoJuego,CincoInicial,equipo,TirosDosAciertos,TirosDosFallos,TirosTresAciertos,TirosTresFallos,TirosLibresAciertos,TirosLibresFallos,plus_minus,posesiones_consumidas,posesiones_jugadas,rebote_of_disp,rebote_def_disp,puntos_q4_y_prorroga,puntos_clutch,posesiones_estimadas,partido_key,TiempoJuego_seg,TiempoJuego_min,nacionalidad,rival,fecha_hora
0,96264,1932,0,"ARIAS, J.","ARIAS, JUAN PABLO",0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,00:00,False,ARGENTINO (J),0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.00,ARGENTINO (J) vs ATENAS (C) (002/04/2025 21:00),0.0,0.000000,Argentino,ATENAS,02/04/2025
1,96269,1932,0,"CAPELLI, S.","CAPELLI, SANTIAGO",0.0,1.0,0.0,1.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0,1.0,06:05,False,ARGENTINO (J),0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.00,ARGENTINO (J) vs ATENAS (C) (002/04/2025 21:00),365.0,6.083333,Argentino,ATENAS,02/04/2025
2,323790,1932,0,"FRONTERA, R.","FRONTERA, RAMIRO JEREMIAS",5.0,6.0,0.0,6.0,2.0,1.0,1.0,1.0,2.0,3.0,2.0,3.0,24:02,False,ARGENTINO (J),2.0,0.0,0.0,0.0,1.0,0.0,-21.0,11.0,48.0,30.0,15.0,0.0,0.0,3.44,ARGENTINO (J) vs ATENAS (C) (002/04/2025 21:00),1442.0,24.033333,Argentino,ATENAS,02/04/2025


Con esta base quiero formar una que tenga las columnas:
- ..._suma (nuericas sumadas)
- ..._prom_minutos ( _suma / minutos_jugados_suma)
- TirosDeCampo_suma = (TirosDosAciertos_suma + TirosDosFallos_suma + TirosTresAciertos_suma + TirosTresFallos_suma)
- TOV% 100 × (Perdidas_suma / posesiones_consumidas_suma)
- eFG%  ((TirosDosAciertos_suma + 1.5 * TirosTresAciertos_suma) / TirosDeCampo_suma)
- ORB% = 100 × (ReboteOfensivo_suma / rebote_of_disp_suma)
- DRB% = 100 × (ReboteDefensivo_suma / rebote_def_disp_suma)
- RB% = 100 × (RebotesTotales_suma /( rebote_of_disp_suma + rebote_def_disp_suma ))
- FTr = (TirosLibresAciertos_suma / TirosDeCampo_suma) 
- TS% = 100 × (Puntos_suma / (2 × (TirosDeCampo_suma + 0.44 × (TirosLibresAciertos + TirosLibresFallos))))
- USG% = 100 × (posesiones_consumidas_suma / posesiones_jugadas_suma)
- AST% = 100 × (Asistencias_suma / ((TiempoJuego_min_suma / (minutos_totales_equipo / 5)) * tiros_campo_anotados_equipo)  - tiros_campo_anotados_jugador) EVALUAR
- AST/TOVr = (Asistencias_suma / Perdidas_suma)
- game_score = Puntos_suma 
                + (0.4 * (TirosDosAciertos_suma + TirosTresAciertos_suma))  
                - (0.7 * TirosDeCampo_suma)
                - (0.4 * (TirosLibresFallos)
                + (0.7 * ReboteOfensivo_suma)
                + (0.3 * ReboteDefensivo_suma)
                + Recuperaciones_suma
                + (0.7 * Asistencias_suma)
                + (0.7 * TaponCometido_suma)
                - (0.4 * FaltaCometida_suma)
                - Perdidas_suma)

## Group By

In [ ]:
# Definir las columnas que son numéricas y no son 'NombreCompleto'
columnas_numericas = combined_df.select_dtypes(include=np.number).columns.drop("NombreCompleto", errors='ignore')
columnas_no_numericas = combined_df.select_dtypes(exclude=np.number).columns.drop("NombreCompleto", errors='ignore')

# Crear el diccionario de agregaciones
agg_dict = {col: 'sum' for col in columnas_numericas}
agg_dict.update({col: 'last' for col in columnas_no_numericas})

# Realizar la agregación
df_players = combined_df.groupby("NombreCompleto", observed=False).agg(agg_dict).reset_index()

# Renombrar las columnas sumadas
df_players = df_players.rename(columns={col: f"{col}_suma" for col in columnas_numericas})

# Agregar la columna CantidadPartidosJugados
cantidad_partidos = combined_df[combined_df["TiempoJuego_seg"] > 0].groupby("NombreCompleto", observed=False).size()
df_players["CantidadPartidosJugados"] = df_players["NombreCompleto"].map(cantidad_partidos).fillna(0).astype(int)
df_players

In [ ]:
# Guardar df_players y combined_df en formato parquet en la carpeta data\processed
# df_players.to_parquet("../data/processed/df_players.parquet", index=False)
# combined_df.to_parquet("../data/processed/combined_df.parquet", index=False)